In [1]:
from clpsych import store, data
import codecs
import os
import pandas

## Read in data from parse files

In [ ]:
#currently unused

tokenized = {}
i = 0

base_i = i
more = True
while more:
    if i % 10 == 0: print i
    if i < base_i + 50 and os.path.isfile('parses/{}.parse'.format(i)):
        docs, titles = read_parses('parses/{}.parse'.format(i))
        keys = [key for key in docs.keys() if key]
        for key in keys:
            doc, title = [], []
            for token in docs[key]:
                doc.append(token)
            for token in titles[key]:
                title.append(token.lemma)
            tokenized[key] = (title, doc)
        i += 1
    else:
        break

## Load in data.h5

In [ ]:
#including parse takes 6 hours and doesn't do anything
config = {
    'mask': './**/**/*.posts',
    'train_mask': './**/**/TRAIN.txt',
    'test_mask': './**/**/TEST.txt',
    'dev_mask': './**/**/DEV.txt',
    'sample_mask': './SAMPLE.txt',
    #'parse':'./parses/*.parse',
    #'parsed':True
}

load = store.Store('data.h5', config=config, overwrite = False)
loaded_data = load.data

## Calculate Lengths

In [2]:
#this can be replaced with read_parses() if it saves "tok.sents" and "tok.rank"

import spacy
nlp = spacy.load('en')

In [3]:

indices = data.read_indices('SAMPLE.txt')
load = store.Store('data.h5')
loaded_data = load.select(indices)

len(loaded_data)

100138

In [6]:
#this can be replaced with read_parses() if it saves "tok.sents"
#currently takes ~3 hours to run


df = pandas.DataFrame(columns=('user_id', 'post_id', 'num_sent', 'avg_len',  'accuracy', 'freq_accuracy', 'len_title', 'accuracy_title', 'freq_title'))
#'spelling', 'freq', 'spelling_title', 
count = 0 
for index, row in loaded_data.iterrows():
    index = count #index, row in itterrows resets every 10k or so!
    if (count%25000 == 0): print "Completed ", count
    #read with spacy, decode into unicode
    spacy_doc = nlp(row['text'].decode("utf-8"))
    spacy_title = nlp(row['title'].decode("utf-8"))
    
    #add tokens if their spacy rank isn't 0 (we could set a threshold here as well)
    no_typos = [tok.text for tok in spacy_doc if tok.rank!=0]    
    infreq = [tok.text for tok in spacy_doc if (tok.rank!=0 and tok.rank<100000)]    
    
    #simple counts (number of sentences, and avg lengh of sentence)
    num_sent = len(list(spacy_doc.sents))
    avg_len_sent =  sum([len(sent) for sent in list(spacy_doc.sents)])/len(list(spacy_doc.sents))  if len(list(spacy_doc.sents))> 0 else 0
    
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy = float(len(no_typos))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    accuracy_infreq = float(len(infreq))/len(spacy_doc) if len(spacy_doc) > 0 else 1
    
    #do the same only for titles
    no_typos_title = [tok.text for tok in spacy_title if tok.rank!=0]   
    infreq_title = [tok.text for tok in spacy_title if (tok.rank!=0 and tok.rank<100000)]
    
    len_title =  len(spacy_title)
    #most appear to have values over 90% (aka little typos).  Chose to set 0 length docs to 1.  
    accuracy_title = float(len(no_typos_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    accuracy_infreq_title = float(len(infreq_title))/len(spacy_title) if len(spacy_title) > 0 else 1
    
    #append to new DF in the same row order
    try:
        df.loc[index] = row['user_id'], row['post_id'], num_sent, avg_len_sent, accuracy, accuracy_infreq, len_title, accuracy_title, accuracy_infreq_title
        #no_typos, infreq,no_typos_title,
    except:
        print "Issue with row ",  index
        df.loc[index] = row['user_id'], row['post_id'], 0, 0, 0, 0, 0, 0
    count+=1
print df
    

Completed  0
Completed  25000
Completed  50000
Completed  75000
Completed  100000
        user_id post_id  user_idnum_sent  avg_len  accuracy  freq_accuracy  \
0       -1014.0   wfp2i              3.0     12.0  0.973684       0.973684   
1       -1014.0   wx3r4              4.0     21.0  1.000000       1.000000   
2       -1014.0   x4hmq              0.0      0.0  1.000000       1.000000   
3       -1014.0   xm156              0.0      0.0  1.000000       1.000000   
4       -1014.0   xo2vm              0.0      0.0  1.000000       1.000000   
5       -1014.0   yooo6              0.0      0.0  1.000000       1.000000   
6       -1014.0   yy9w6              0.0      0.0  1.000000       1.000000   
7       -1014.0   z4k8a              0.0      0.0  1.000000       1.000000   
8       -1014.0   zg1e5              0.0      0.0  1.000000       1.000000   
9       -1014.0  102okc              0.0      0.0  1.000000       1.000000   
10      -1014.0  11xmvh              0.0      0.0  1.000000 

In [8]:
df[['user_id','post_id', 'avg_len', 'accuracy', 'freq_accuracy','len_title','user_idnum_sent','freq_title','accuracy_title']].to_csv('sample_csv.csv')

In [ ]:
load = store.Store('data_Denis.h5', config=config, overwrite = False)
data = load.data

In [ ]:
#doc
load.create_document_feature("avg_len", df[['post_id', 'avg_len']])
load.create_document_feature("num_sent", df[['post_id', 'num_sent']])
load.create_document_feature("typo_acc", df[['post_id', 'accuracy']])
load.create_document_feature("freq_acc", df[['post_id', 'freq_accuracy']])

#title
load.create_document_feature("len_title", df[['post_id', 'len_title']])
load.create_document_feature("typo_acc_title", df[['post_id', 'accuracy_title']])
load.create_document_feature("freq_acc_title", df[['post_id', 'freq_title']])


load.finalize
load.documents

In [ ]:
load.documents.to_csv("full_csv.csv")

indices = data.read_indices('SAMPLE.txt')
load = store.Store('data.h5')
df = load.select(indices, df=load.documents)
df[['user_id','post_id','subreddit','avg_len', 'accuracy', 'freq_accuracy','len_title','num_sent','freq_title','accuracy_title']].to_csv('sample_csv.csv')


In [ ]:
#code for converting spelling into a long string.  This could be written into a DF if needed
storage = []
for row in df['spelling'][0:10]:
    string = ""
    for tok in row:
        string = string+" " + tok.text
    storage.append(string)

## Potpourri Analysis

In [3]:
#can we trust .rank?  I think yes.  High ranking words are pronouns, punctuation.  Low ranking words are proper nouns.  Misspellings are generally 0s (somethines high numbers)
words = nlp(u'suicide death die I ugh hate myself me wtf asfab beelzebub shakespeare javert misppeling reddit')

[tok.rank for tok in words]


[2594, 736, 683, 4, 8007, 405, 355, 49, 3075, 0, 264328, 80517, 436651, 0, 465]